In [3]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout
import pandas as pd

In [2]:
pre = "/Users/weilu/Research/server/jun_week1_2020/protein_DNA_benchmark/"
fileLocation = f"{pre}/DNAProtein_Platform_OpenCL_date_20200226_pdb_1a36_repetition_0_clean.pdb"
pdb = PDBFile(fileLocation)

In [4]:
res_list = list(pdb.topology.residues())

In [6]:
res_list = list(pdb.topology.residues())
atom_list = list(pdb.topology.atoms())
protein_resNames = ["NGP", "IGL", "IPR", "NTER", "CTER"]
DNA_resNames = ["DA", "DC", "DT", "DG"]
protein_res_list = []
DNA_res_list = []
ligand_res_list = []
for res in res_list:
    if res.name in protein_resNames:
        protein_res_list.append(res)
    elif res.name in DNA_resNames:
        DNA_res_list.append(res)
    else:
        ligand_res_list.append(res)

protein_atom_list = []
DNA_atom_list = []
ligand_atom_list = []
for atom in atom_list:
    if atom.residue.name in protein_resNames:
        protein_atom_list.append(atom)
    elif atom.residue.name in DNA_resNames:
        DNA_atom_list.append(atom)
    else:
        ligand_atom_list.append(atom)



In [38]:
a = DNA_atom_list[0]

In [45]:
a.index

0

In [51]:
a = (atom1_pos - atom2_pos)

In [53]:
def compute_dis(pos1, pos2):
    dis = pos1 - pos2
    dis = dis.value_in_unit(nanometer)
    r = (dis[0]**2 + dis[1]**2 + dis[2]**2)**0.5
    return r

In [4]:
import sys

sys.path.append("/projects/pw8/wl45/open3spn2")
sys.path.append("/projects/pw8/wl45/openawsem")
sys.path.append("/Users/weilu/open3spn2")
sys.path.append("/Users/weilu/openmmawsem")
sys.path.append("C:/Users/luwei/Documents/GitHub/open3spn2")
sys.path.append("C:/Users/luwei/Documents/GitHub/openawsem")

import open3SPN2

In [60]:
dna=open3SPN2.DNA.fromCoarsePDB(fileLocation)

In [85]:
bonds_list = []
interacting_atom = ["A", "C", "G", "T"]
a = data.query("resname in @DNA_resNames").query("name in @interacting_atom").reset_index(drop=True)
n_atom = len(a)
for i in range(n_atom):
    dna_res_1 = a.iloc[i]
    chainID_1 = dna_res_1["chainID"]
    for j in range(i+1, n_atom):
        dna_res_2 = a.iloc[j]
        chainID_2 = dna_res_2["chainID"]
        if chainID_1 == chainID_2:
            continue
        atom1_pos = dna_res_1[["x", "y", "z"]].values
        atom2_pos = dna_res_2[["x", "y", "z"]].values
        dis = atom1_pos - atom2_pos
        r = (dis[0]**2 + dis[1]**2 + dis[2]**2)**0.5
        if r > 6.5:
            continue
        # print(i, j, r)
        bonds_list.append([dna_res_1["serial"], dna_res_2["serial"], r])

In [ ]:
for (atom1, atom2, r) in bonds_list:
    print(atom1, atom2, r)

In [72]:
b = a.iloc[1][["x", "y", "z"]].values

In [93]:
from open3SPN2 import function_terms

In [95]:
function_terms.dna_constraint_term()

TypeError: dna_constraint_term() missing 1 required positional argument: 'dna'

In [56]:
dna_res_n = len(DNA_res_list)
for i in range(dna_res_n):
    dna_res_1 = DNA_res_list[i]
    chainID_1 = dna_res_1.chain.id
    for j in range(i+1, dna_res_n):
        dna_res_2 = DNA_res_list[j]
        chainID_2 = dna_res_2.chain.id
        if chainID_1 == chainID_2:
            continue
        atom_1 = get_interacting_DNA_atom(dna_res_1)
        atom_2 = get_interacting_DNA_atom(dna_res_2)
        atom1_pos = pdb.positions[atom_1.index]
        atom2_pos = pdb.positions[atom_2.index]
        r = compute_dis(atom1_pos, atom2_pos)
        if r > 0.65:
            continue
        print(i, dna_res_1, j, dna_res_2, r)

1 <Residue 1 (DA) of chain 0> 41 <Residue 41 (DT) of chain 1> 0.6469376708771873
1 <Residue 1 (DA) of chain 0> 42 <Residue 42 (DT) of chain 1> 0.5491159258298746
2 <Residue 2 (DA) of chain 0> 40 <Residue 40 (DT) of chain 1> 0.6349142776784908
2 <Residue 2 (DA) of chain 0> 41 <Residue 41 (DT) of chain 1> 0.5734785785014116
2 <Residue 2 (DA) of chain 0> 42 <Residue 42 (DT) of chain 1> 0.6200130724428318
3 <Residue 3 (DA) of chain 0> 40 <Residue 40 (DT) of chain 1> 0.5904393025536154
4 <Residue 4 (DA) of chain 0> 39 <Residue 39 (DT) of chain 1> 0.5754847174339213
4 <Residue 4 (DA) of chain 0> 40 <Residue 40 (DT) of chain 1> 0.5720431714477502
5 <Residue 5 (DG) of chain 0> 38 <Residue 38 (DC) of chain 1> 0.5760794563252537
5 <Residue 5 (DG) of chain 0> 39 <Residue 39 (DT) of chain 1> 0.5544115168356443
6 <Residue 6 (DA) of chain 0> 36 <Residue 36 (DG) of chain 1> 0.6052784483194491
6 <Residue 6 (DA) of chain 0> 37 <Residue 37 (DT) of chain 1> 0.5796950922683405
6 <Residue 6 (DA) of chain 0

In [34]:
def get_interacting_DNA_atom(DNA_res):
    for atom in DNA_res.atoms():
        # print(atom)
        if atom.name in ["A", "T", "C", "G"]:
            return atom
        

In [12]:
list(a.atoms())

[<Atom 127 (T) of chain 1 residue 43 (DT)>,
 <Atom 128 (P) of chain 1 residue 43 (DT)>,
 <Atom 129 (S) of chain 1 residue 43 (DT)>]

In [29]:
for atom in a.atoms():
    print(atom)
    if atom.name not in ["A", "T", "C", "G"]:
        continue
    for 
    

<Atom 127 (T) of chain 1 residue 43 (DT)>
<Atom 128 (P) of chain 1 residue 43 (DT)>
<Atom 129 (S) of chain 1 residue 43 (DT)>


In [36]:
atom - atom

TypeError: unsupported operand type(s) for -: 'Atom' and 'Atom'

In [30]:
atom.id

'129'

In [31]:
atom.index

129

'S'

In [17]:
dna_name = a.name[1]

In [18]:
a[dna_name]

TypeError: 'Residue' object is not subscriptable

In [28]:
a

<Residue 43 (DT) of chain 1>

In [25]:
list(a.external_bonds())

[]

In [10]:
a.chain.id

'B'

In [5]:
res_list

[<Residue 0 (DA) of chain 0>,
 <Residue 1 (DA) of chain 0>,
 <Residue 2 (DA) of chain 0>,
 <Residue 3 (DA) of chain 0>,
 <Residue 4 (DA) of chain 0>,
 <Residue 5 (DG) of chain 0>,
 <Residue 6 (DA) of chain 0>,
 <Residue 7 (DC) of chain 0>,
 <Residue 8 (DT) of chain 0>,
 <Residue 9 (DT) of chain 0>,
 <Residue 10 (DA) of chain 0>,
 <Residue 11 (DG) of chain 0>,
 <Residue 12 (DA) of chain 0>,
 <Residue 13 (DA) of chain 0>,
 <Residue 14 (DA) of chain 0>,
 <Residue 15 (DA) of chain 0>,
 <Residue 16 (DA) of chain 0>,
 <Residue 17 (DT) of chain 0>,
 <Residue 18 (DT) of chain 0>,
 <Residue 19 (DT) of chain 0>,
 <Residue 20 (DT) of chain 0>,
 <Residue 21 (DT) of chain 0>,
 <Residue 22 (DA) of chain 1>,
 <Residue 23 (DA) of chain 1>,
 <Residue 24 (DA) of chain 1>,
 <Residue 25 (DA) of chain 1>,
 <Residue 26 (DA) of chain 1>,
 <Residue 27 (DT) of chain 1>,
 <Residue 28 (DT) of chain 1>,
 <Residue 29 (DT) of chain 1>,
 <Residue 30 (DT) of chain 1>,
 <Residue 31 (DT) of chain 1>,
 <Residue 32 (DC) 

In [5]:
import os
import argparse
import sys
import openmmawsem
import helperFunctions.myFunctions

In [6]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout
from pdbfixer import *

In [7]:
chains_to_simulate = "B"

In [8]:
removeHeterogens = False

In [15]:
pdb_filename = "/Users/weilu/Research/server/jun_week1_2020/no_DNA_protein_DNA_benchmark/cleaned_pdbs/1a36.pdb"


fixer = PDBFixer(filename=pdb_filename)
# we only want pdb chains, ligands or DNA chain will be ignored here.
fixer.removeHeterogens(keepWater=False)
# remove unwanted chains
chains = list(fixer.topology.chains())

In [16]:
chains

[<Chain 0>, <Chain 1>, <Chain 2>]

In [17]:
for c in chains:
    print(c.id)

A
B
C


In [21]:
a = list(c.residues())

In [22]:
b = a[0]

In [23]:
b.id

'1'

In [27]:
import numpy as np

In [29]:
if np.alltrue([a.name in dnaResidues for a in c.residues()]) is True:
    # this chain is a DNA chain
    

False

In [26]:
b.name in dnaResidues

False

In [ ]:
proteinResidues = ['ALA', 'ASN', 'CYS', 'GLU', 'HIS', 'LEU', 'MET', 'PRO', 'THR', 'TYR', 'ARG', 'ASP', 'GLN', 'GLY', 'ILE', 'LYS', 'PHE', 'SER', 'TRP', 'VAL']
rnaResidues = ['A', 'G', 'C', 'U', 'I']
dnaResidues = ['DA', 'DG', 'DC', 'DT', 'DI']

In [25]:
dnaResidues = ['DA', 'DG', 'DC', 'DT', 'DI']

In [49]:
chains = list(fixer.topology.chains())
a = ""

proteinResidues = ['ALA', 'ASN', 'CYS', 'GLU', 'HIS', 'LEU', 'MET', 'PRO', 'THR', 'TYR', 'ARG', 'ASP', 'GLN', 'GLY', 'ILE', 'LYS', 'PHE', 'SER', 'TRP', 'VAL']
rnaResidues = ['A', 'G', 'C', 'U', 'I']
dnaResidues = ['DA', 'DG', 'DC', 'DT', 'DI']
for c in chains:
    # print([a.name in dnaResidues for a in c.residues()])
    if True and np.alltrue([a.name in dnaResidues for a in c.residues()]):
        continue
    if c.id in 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789':
        a += c.id

In [50]:
a

'C'

In [12]:
pdb_filename = "/Users/weilu/Research/server/jun_week1_2020/no_DNA_protein_DNA_benchmark/cleaned_pdbs/1a36.pdb"
fixer = PDBFixer(filename=pdb_filename)

# remove unwanted chains
chains = list(fixer.topology.chains())
chains_to_remove = [i for i, x in enumerate(chains) if x.id not in chains_to_simulate]
fixer.removeChains(chains_to_remove)

#Identify Missing Residues
fixer.findMissingResidues()
fixer.missingResidues = {}

#Replace Nonstandard Residues
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()

#Remove Heterogens
if removeHeterogens:
    fixer.removeHeterogens(keepWater=False)

#Add Missing Heavy Atoms
fixer.findMissingAtoms()
fixer.addMissingAtoms()

#Add Missing Hydrogens
fixer.addMissingHydrogens(7.0)
# PDBFile.writeFile(fixer.topology, fixer.positions, open(cleaned_pdb_filename, 'w'), keepIds=keepIds)

#Read sequence
# structure = PDBParser().get_structure('X', cleaned_pdb_filename)


In [13]:
fixer.topology

<Topology; 1 chains, 22 residues, 700 atoms, 1246 bonds>